In [ ]:
!pip install segmentation_models_pytorch
!pip install wandb
!pip install torcheval
!pip install torchmetrics

# Iniatial

In [2]:
!mkdir Master
!mkdir /content/Master/instances
!mkdir /content/Master/Output

# !wget -P Master http://images.cocodataset.org/zips/val2017.zip
!wget -P Master http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip
!wget -P /content/Master/instances http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget -P Master http://images.cocodataset.org/zips/train2017.zip

--2024-05-31 08:02:25--  http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.29.16, 52.217.99.44, 52.216.30.124, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.29.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 860725834 (821M) [application/zip]
Saving to: ‘Master/panoptic_annotations_trainval2017.zip’

panoptic_annotation 100%[===================>] 820.85M  40.7MB/s    in 21s     

2024-05-31 08:02:46 (39.8 MB/s) - ‘Master/panoptic_annotations_trainval2017.zip’ saved [860725834/860725834]

--2024-05-31 08:02:46--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.143, 3.5.28.19, 3.5.25.246, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.143|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [

In [3]:
# !unzip /content/Master/val2017.zip -d Master
!unzip /content/Master/panoptic_annotations_trainval2017.zip -d Master
!unzip /content/Master/instances/annotations_trainval2017.zip -d /content/Master/instances
!unzip /content/Master/train2017.zip -d Master

Streaming output truncated to the last 5000 lines.
 extracting: Master/train2017/000000259556.jpg  
 extracting: Master/train2017/000000381931.jpg  
 extracting: Master/train2017/000000569592.jpg  
 extracting: Master/train2017/000000229396.jpg  
 extracting: Master/train2017/000000488990.jpg  
 extracting: Master/train2017/000000348684.jpg  
 extracting: Master/train2017/000000234031.jpg  
 extracting: Master/train2017/000000563584.jpg  
 extracting: Master/train2017/000000276069.jpg  
 extracting: Master/train2017/000000401194.jpg  
 extracting: Master/train2017/000000502089.jpg  
 extracting: Master/train2017/000000192183.jpg  
 extracting: Master/train2017/000000425127.jpg  
 extracting: Master/train2017/000000126766.jpg  
 extracting: Master/train2017/000000324161.jpg  
 extracting: Master/train2017/000000452746.jpg  
 extracting: Master/train2017/000000423782.jpg  
 extracting: Master/train2017/000000546343.jpg  
 extracting: Master/train2017/000000249290.jpg  
 extracting: Maste

In [4]:
# !unzip /content/Master/annotations/panoptic_val2017.zip -d Master
# !mv /content/Master/annotations/panoptic_val2017.json /content/Master # move src dest
# !mv /content/Master/instances/annotations/instances_val2017.json /content/Master # move src dest
# # !unzip /content/Master/panoptic_annotations_trainval2017.zip -d Master

!rm -r /content/Master/panoptic_annotations_trainval2017.zip
!rm -r /content/Master/train2017.zip

!unzip /content/Master/annotations/panoptic_train2017.zip -d Master
!mv /content/Master/annotations/panoptic_train2017.json /content/Master # move src dest
!mv /content/Master/instances/annotations/instances_train2017.json /content/Master # move src dest
# !unzip /content/Master/panoptic_annotations_trainval2017.zip -d Master

Streaming output truncated to the last 5000 lines.
  inflating: Master/panoptic_train2017/000000152038.png  
  inflating: Master/panoptic_train2017/000000008336.png  
  inflating: Master/panoptic_train2017/000000279563.png  
  inflating: Master/panoptic_train2017/000000188522.png  
  inflating: Master/panoptic_train2017/000000473573.png  
  inflating: Master/panoptic_train2017/000000531032.png  
  inflating: Master/panoptic_train2017/000000267985.png  
  inflating: Master/panoptic_train2017/000000334311.png  
  inflating: Master/panoptic_train2017/000000187611.png  
  inflating: Master/panoptic_train2017/000000011667.png  
  inflating: Master/panoptic_train2017/000000497187.png  
  inflating: Master/panoptic_train2017/000000218189.png  
  inflating: Master/panoptic_train2017/000000030871.png  
  inflating: Master/panoptic_train2017/000000266443.png  
  inflating: Master/panoptic_train2017/000000567274.png  
  inflating: Master/panoptic_train2017/000000521221.png  
  inflating: Master/p

In [5]:
!rm -r /content/Master/val2017.zip
# !rm -r /content/Master/panoptic_annotations_trainval2017.zip
# !rm -r /content/Master/train2017.zip

import shutil

shutil.rmtree('/content/Master/annotations', ignore_errors=True)
shutil.rmtree('/content/Master/instances', ignore_errors=True)
shutil.rmtree('/content/Master/__MACOSX', ignore_errors=True)

rm: cannot remove '/content/Master/val2017.zip': No such file or directory


In [6]:
# import shutil

# shutil.rmtree('/content/Proiect-Dizertatie', ignore_errors=True)
# shutil.rmtree('/content/Master/instances', ignore_errors=True)

# Libraries

In [1]:
# For managing COCO dataset
from pycocotools.coco import COCO

# For creating and managing folder/ files
import glob
import os
import shutil

# For managing images
from PIL import Image
import skimage.io as io

# Basic libraries
import numpy as np
import pandas as pd
import random
import cv2

# For plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import wandb

# For importing models and working with them
## Torch
import torch
import torch.utils.data # for Dataset
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp

## Torchvision
import torchvision
from torchvision.transforms import transforms

# For creating train - test splits
from sklearn.model_selection import train_test_split

import pathlib
import pylab
import requests
from io import BytesIO
from pprint import pprint
from tqdm import tqdm
import time
from imutils import paths

# Performance Metrics
from sklearn.metrics import multilabel_confusion_matrix


# Import Files
import BinaryMasks as bm
import CalcMetrics as cm
import CreateDataset as crd
import FilterImgs as flt
import TrainModel as trModel
import ValidateModel as valModel
import TestModel as testModel
import SaveCSV
import DiceLoss as dcloss
import EarlyStopping as stopping
import PlotResults as pr
import MainLoop as main
import DisplayMetrics as dm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# %matplotlib inline

cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu
cpu


# Configuration

In [2]:
data = 'train2017'

# Data and Masks
IMAGE_PATH = os.path.join('/content/Master/', data)
MASK_PATH = os.path.join('/content/Master/', 'panoptic_' + data)

# define the path to the base output directory
BASE_OUTPUT = '/content/Master/Output'

# define the path to the output serialized model, model training plot, and testing image paths
MODEL_PATH = os.path.join(BASE_OUTPUT, "unet_tgs_salt.pth")
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])



# determine if we will be pinning memory during data loading
PIN_MEMORY = True if device == "cuda" else False


# define the number of channels in the input, number of classes, and number of levels in the U-Net model
# NUM_CHANNELS = 3
# NUM_CLASSES = 2 # person vs Background

# define the input image dimensions
INPUT_IMAGE_WIDTH = 256
INPUT_IMAGE_HEIGHT = 256

# define threshold to filter weak predictions
THRESHOLD = 0.5 # for binary classification

# Data

## Data Preparation

In [9]:
# Prepare Directory for Binary Masks
shutil.rmtree('/content/Master/Binary_Masks', ignore_errors = True)

!mkdir /content/Master/Binary_Masks

ann_path = '/content/Master/instances_train2017.json'
filterCls = ['person']
bm_path = '/content/Master/Binary_Masks/'

small = 9000
medium = 15000
large = 30000

imagePaths, maskPaths, binaryMaskPaths = flt.filter_data(ann_path, filterCls, bm_path, IMAGE_PATH, MASK_PATH)
SaveCSV.create_csvs(imagePaths, binaryMaskPaths, small, medium, large)

loading annotations into memory...
Done (t=30.35s)
creating index...
index created!
Number of images containing all the classes: 64115
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000


In [ ]:
# IMG_PATHS = os.path.sep.join([BASE_OUTPUT, "img_paths.txt"])
# BMASKS_PATHS = os.path.sep.join([BASE_OUTPUT, "bmasks_paths.txt"])

# print("[INFO] saving image paths...")
# f = open(IMG_PATHS, "w")
# f.write("\n".join(imagePaths))
# f.close()

# print("[INFO] saving binary masks paths...")
# f = open(BMASKS_PATHS, "w")
# f.write("\n".join(binaryMaskPaths))
# f.close()

[INFO] saving image paths...
[INFO] saving binary masks paths...


## Train

In [ ]:
# https://github.com/qubvel/segmentation_models.pytorch
# https://github.com/qubvel/segmentation_models.pytorch#architectures

RESNET_SIZE = 256
transf_size = RESNET_SIZE

# Initialize transformations - train
transforms_train = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

# Initialize transformations - test (optional)
transforms_test = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])


# Define hyperparameters
INIT_LR = 0.0001
BATCH_SIZE = 32
NUM_EPOCHS = [10, 20, 50, 100, 150]
TEST_SPLIT = [0.1, 0.2] # 0.3
VALID_SPLIT = 0.1 # 0.1

wb_name = "Resnet_Unet_MSCOCO"
loaded_df = pd.read_csv(r'shuffled_df_medium.csv')

# 73c482950d37419655b401f882c34c719ca587af

main.main_loop(loaded_df, TEST_PATHS, transforms_train, transforms_test, wb_name, INIT_LR, BATCH_SIZE, NUM_EPOCHS, TEST_SPLIT, VALID_SPLIT, PIN_MEMORY)





wandb: Currently logged in as: bianca-apostolescu (bianca-team). Use `wandb login --relogin` to force relogin


[INFO] TEST_SPLIT = 0.1 ...
Splits, Datasets, and Dataloaders
[INFO] saving testing image paths...
[INFO] Total time taken to create the dataset and dataloader: 0.01s
trainSteps = 379, testSteps = 46, valSteps = 42


[INFO] Training the network for 10 epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Training...
